In [1]:
import pandas as pd
from sklearn.preprocessing import MaxAbsScaler

# read datasets
train_df = pd.read_csv('data/train2.csv')
test_df = pd.read_csv('data/test2.csv')

y1_name, y2_name, y3_name = "dir_costs", "traffic_costs_s_r", "lost_trips_costs_s_r"
train_y1, train_y2, train_y3 = train_df[y1_name], train_df[y2_name], train_df[y3_name]
test_y1, test_y2, test_y3 = test_df[y1_name], test_df[y2_name], test_df[y3_name]

# scale features
X_train = train_df.drop(columns=[y1_name, y2_name, y3_name])
scaler = MaxAbsScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

X_test = test_df.drop(columns=[y1_name, y2_name, y3_name])
X_test_scaled = scaler.transform(X_test)


<h2> Pick arbitrary sigma</h2>

In [2]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y1 = GRNN(sigma=0.1)
grnn_y1.fit(X_train_scaled, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, grnn_y1.predict(X_test_scaled))


------ test metrics ------
Mean Squared Error (MSE):              63742646669581.4765625000
Root Mean Squared Error (RMSE):        7983899.2146432735
Mean Absolute Error (MAE):             5571384.4635659913
R-squared (R²):                        0.6810155400
Mean Absolute Percentage Error (MAPE): 0.1479991331
Max Error (ME):                        35165377.4447595924
Median Absolute Error (MedAE):         3873115.1668858230


<h2> Use differential evolution to pick optimal sigma <h2>

In [3]:
from sklearn.metrics import r2_score
from scipy.optimize import differential_evolution

def objective_function(sigma, train_X, train_y, test_X, test_y):
    grnn = GRNN(sigma=sigma[0])
    grnn.fit(train_X, train_y)
    pred_y = grnn.predict(test_X)
    r2 = r2_score(test_y, pred_y)
    return -r2

<h1> Y1 </h1>

In [5]:
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(0, 10)], 
                                          updating='deferred', 
                                          workers=10,
                                          disp=True,
                                          tol=0.0000001,
                                          atol=0.0000001,
                                          args=(X_train_scaled, train_y1, X_test_scaled, test_y1))
print(optimization_res)
print("Optimal sigma_y1: {:.{}f}".format(optimization_res.x[0], 9))

differential_evolution step 1: f(x)= -0.632792
differential_evolution step 2: f(x)= -0.632792
differential_evolution step 3: f(x)= -0.632792
differential_evolution step 4: f(x)= -0.681187
differential_evolution step 5: f(x)= -0.681187
differential_evolution step 6: f(x)= -0.681187
differential_evolution step 7: f(x)= -0.681187
differential_evolution step 8: f(x)= -0.681187
differential_evolution step 9: f(x)= -0.681187
differential_evolution step 10: f(x)= -0.681187
differential_evolution step 11: f(x)= -0.681187
differential_evolution step 12: f(x)= -0.681187
differential_evolution step 13: f(x)= -0.681187
differential_evolution step 14: f(x)= -0.681187
differential_evolution step 15: f(x)= -0.681188
differential_evolution step 16: f(x)= -0.681188
differential_evolution step 17: f(x)= -0.681188
differential_evolution step 18: f(x)= -0.681188
differential_evolution step 19: f(x)= -0.681188
differential_evolution step 20: f(x)= -0.681188
differential_evolution step 21: f(x)= -0.681188
P

In [6]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y1 = GRNN(sigma=0.097183859)
grnn_y1.fit(X_train_scaled, train_y1)

print("------ test metrics ------")
print_metrics(test_y1, grnn_y1.predict(X_test_scaled))

------ test metrics ------
Mean Squared Error (MSE):              63708187790534.0781250000
Root Mean Squared Error (RMSE):        7981740.8997369790
Mean Absolute Error (MAE):             5549585.5889385771
R-squared (R²):                        0.6811879810
Mean Absolute Percentage Error (MAPE): 0.1470635573
Max Error (ME):                        35498167.9150355011
Median Absolute Error (MedAE):         3861312.4261567965


<h1> Y2 </h1>

In [89]:
optimization_res = differential_evolution(func=objective_function, 
                                          bounds=[(0, 10)], 
                                          updating='deferred', 
                                          workers=10,
                                          disp=True,
                                          tol=0.0000001,
                                          atol=0.0000001,
                                          args=(X_train_scaled, train_y2, X_test_scaled, test_y2))
print(optimization_res)
print("Optimal sigma_y2: {:.{}f}".format(optimization_res.x[0], 9))

 message: Optimization terminated successfully.
 success: True
     fun: -0.7023818942348741
       x: [ 6.106e-02]
     nit: 11
    nfev: 190
     jac: [-2.220e-08]
Optimal sigma_y1: 0.061057788


In [90]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y2 = GRNN(sigma=0.061057788)
grnn_y2.fit(X_train_scaled, train_y2)

print("------ test metrics ------")
print_metrics(test_y2, grnn_y2.predict(X_test_scaled))


------ test metrics ------
Mean Squared Error (MSE):              68501419241731.3984375000
Root Mean Squared Error (RMSE):        8276558.4177078940
Mean Absolute Error (MAE):             4388624.5701665469
R-squared (R²):                        0.7023818942
Mean Absolute Percentage Error (MAPE): 1.6495639019
Max Error (ME):                        85655608.1953858286
Median Absolute Error (MedAE):         2658237.9371098289
------ train metrics ------
Mean Squared Error (MSE):              3350194761325.9506835938
Root Mean Squared Error (RMSE):        1830353.7257388122
Mean Absolute Error (MAE):             1022780.9595428550
R-squared (R²):                        0.9845427978
Mean Absolute Percentage Error (MAPE): 0.4284010169
Max Error (ME):                        24835440.3675847687
Median Absolute Error (MedAE):         618150.4338244664


<h1> Y3</h1>

In [91]:
optimization_res = differential_evolution(func=objective_function, bounds=[(0, 10)], updating='deferred', workers=10,
                                        args=(X_train_scaled, train_y3, X_test_scaled, test_y3))
print(optimization_res)
print("Optimal sigma_y3: {:.{}f}".format(optimization_res.x[0], 9))

 message: Optimization terminated successfully.
 success: True
     fun: -0.5353204368090128
       x: [ 8.154e-02]
     nit: 13
    nfev: 218
     jac: [ 9.104e-07]
Optimal sigma_y2: 0.081542483


In [92]:
from GRNN import GRNN
from metrics import print_metrics

grnn_y3 = GRNN(sigma=0.081542483)
grnn_y3.fit(X_train_scaled, train_y3)

print("------ test metrics ------")
print_metrics(test_y3, grnn_y3.predict(X_test_scaled))


------ test metrics ------
Mean Squared Error (MSE):              479167494670487.4375000000
Root Mean Squared Error (RMSE):        21889894.8072047047
Mean Absolute Error (MAE):             11919462.5806300901
R-squared (R²):                        0.5353204368
Mean Absolute Percentage Error (MAPE): 0.1719544801
Max Error (ME):                        200235208.9786968231
Median Absolute Error (MedAE):         6931585.6261113454
------ train metrics ------
Mean Squared Error (MSE):              172358645450598.6562500000
Root Mean Squared Error (RMSE):        13128543.1579668671
Mean Absolute Error (MAE):             7678646.3210368101
R-squared (R²):                        0.8276829904
Mean Absolute Percentage Error (MAPE): 0.1239887136
Max Error (ME):                        90314658.3416012973
Median Absolute Error (MedAE):         4924903.9227655530
